In [2]:
import psycopg2
import paramiko
import pandas as pd
from paramiko import SSHClient
from os.path import expanduser
# from MySQLdb import _mysql
import pymysql as MySQLdb
import psycopg2 as pg


In [3]:
import json
import numpy as np
import time
import requests

In [55]:
import psutil
import os
import datetime
from datetime import timedelta
from io import StringIO
import io
from dateutil import parser
from datetime import datetime


from sshtunnel import SSHTunnelForwarder
import pymysql
import paramiko
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder
from os.path import expanduser
import pymysql as MySQLdb

from flask import Flask, request


In [119]:
import psycopg2 as pg

def upsert_into_table(conn, df, table, idx):
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join([f'"{col}"' for col in df.columns])  # Ensure column names are quoted
    
    updated_cols = ', '.join([f'"{col}"=EXCLUDED."{col}"' for col in df.columns[1:]])
    placeholders = ', '.join(["%s"] * len(df.columns))
    idx_cols = ', '.join([f'"{col}"' for col in idx])  # Ensure index column names are quoted

    query = f"""
        INSERT INTO {table} ({cols}) VALUES ({placeholders})
        ON CONFLICT ({idx_cols}) DO UPDATE SET {updated_cols}
    """

    try:
        with conn.cursor() as cursor:
            cursor.executemany(query, tuples)
            conn.commit()
        print(f"[UPSERT DONE] {table}")
    except (Exception, pg.DatabaseError) as error:
        print(f"Error: {error}")
        conn.rollback()
        return 1

    return 0



def insert_into_table(conn, df, table):
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join([f'"{col}"' for col in df.columns])  # Ensure column names are quoted
    
    placeholders = ', '.join(["%s"] * len(df.columns))
    query = f"INSERT INTO {table} ({cols}) VALUES ({placeholders})"

    try:
        with conn.cursor() as cursor:
            cursor.executemany(query, tuples)
            conn.commit()
        print(f"[INSERT DONE] {table}")
    except (Exception, pg.DatabaseError) as error:
        print(f"Error: {error}")
        conn.rollback()
        return 1

    return 0


def get_process_memory():
    process = psutil.Process(os.getpid())
    return (process.memory_info().rss/1000000)

# curr_date = (datetime.now())
# print("[SCRIPT RUNNING ON] : ", curr_date)

# if len(sys.argv) == 3:
#     start_date = sys.argv[1]
#     end_date = sys.argv[2]
# elif len(sys.argv) > 1:
#     start_date = sys.argv[1] + " 00:00:00"
#     end_date = sys.argv[1] + " 23:59:59"
# else:
#     start_date = ((datetime.now() - timedelta(days=2)).strftime("%Y-%m-%d"))
#     end_date = ((datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d"))

# print("[START DATE AND END DATE] : ", start_date," ", end_date)



memory_start = get_process_memory()

## **dim_payroll**

In [112]:
params = {
    'dbname': "champydwh",
    'user': "jeannetaoliviasantoso",
    'password': "",
    'host': 'localhost',
    'port': 5432
    # 'port': server.local_bind_port

}

conn = psycopg2.connect(**params) 

In [113]:
curs = conn.cursor()
print ("database connected")

database connected


In [114]:


query = f"""
SELECT 
    "EmployeeID", 
    "PaymentDate"
FROM data_payroll_champy_raw;

"""

cursor = conn.cursor()

cursor.execute(query)
row_arr = cursor.fetchall()

In [115]:
df = pd.read_sql_query(query, conn)

/var/folders/6b/fc_hx84s2h99scnnc9qdmz9h0000gn/T/ipykernel_67092/1553155693.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [116]:
df

,EmployeeID,PaymentDate
0,102,2024-09-01
1,103,2024-09-01
2,104,2024-09-01
3,105,2024-08-30
4,107,2024-09-03
...,...,...
551,1194,2024-08-30
552,1195,2024-09-10
553,1197,2024-08-19
554,1198,2024-08-26


In [117]:
# accumulated_df_raw = accumulated_df_raw.replace([np.nan, np.inf, -np.inf], None)


In [121]:

# Connection params
params = {
    'dbname': "champydwh",
    'user': "jeannetaoliviasantoso",
    'password': "",
    'host': 'localhost',
    'port': 5432
}

# Connect to PostgreSQL
conn = psycopg2.connect(**params)

# Insert data
error_status = insert_into_table(conn, df, 'dim_payroll')

# Upsert data (if needed)
# error_status = upsert_into_table_manual(conn, df, 'dim_payroll', ["EmployeeID", "PaymentDate"])

# Close connection
conn.close()


[INSERT DONE] dim_payroll


## **dim_payroll**

In [63]:
params = {
    'dbname': "champydwh",
    'user': "jeannetaoliviasantoso",
    'password': "",
    'host': 'localhost',
    'port': 5432
    # 'port': server.local_bind_port

}

conn = psycopg2.connect(**params) 
curs = conn.cursor()
print ("database connected")

database connected


In [64]:


query = f"""
SELECT 
    "EmployeeID", 
    "ReviewPeriod", 
    "Rating", 
    "Comments", 
    'SAMPLE' as "Prediction"
FROM data_performance_champy_raw;


"""

cursor = conn.cursor()

cursor.execute(query)
row_arr = cursor.fetchall()

In [65]:
df_dim_payroll = pd.read_sql_query(query, conn)

/var/folders/6b/fc_hx84s2h99scnnc9qdmz9h0000gn/T/ipykernel_67092/1183641329.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_dim_payroll = pd.read_sql_query(query, conn)


In [66]:
df_dim_payroll.head(5)

,EmployeeID,ReviewPeriod,Rating,Comments,Prediction
0,102,Q2 2023,2.6,Good performance,SAMPLE
1,103,Q3 2023,1.5,Good performance,SAMPLE
2,104,Q4 2023,4.2,Good performance,SAMPLE
3,105,Q1 2023,4.7,Needs improvement,SAMPLE
4,107,Q3 2023,4.7,Excellent performance,SAMPLE


In [93]:

# Connection params
params = {
    'dbname': "champydwh",
    'user': "jeannetaoliviasantoso",
    'password': "",
    'host': 'localhost',
    'port': 5432
}

# Connect to PostgreSQL
conn = psycopg2.connect(**params)

# Insert data
error_status = insert_into_table(conn, df_dim_payroll[:2], 'dim_performance')

# Upsert data (if needed)
error_status = upsert_into_table(conn, df, 'dim_performance', ['EmployeeID', 'PaymentDate'])

# Close connection
conn.close()


Error: column "PaymentDate" of relation "dim_performance" does not exist
LINE 1: INSERT INTO dim_performance ("EmployeeID","PaymentDate") VAL...
                                                  ^

Error: column "PaymentDate" of relation "dim_performance" does not exist
LINE 1: INSERT INTO dim_performance ("EmployeeID","PaymentDate") VAL...
                                                  ^



In [70]:
df_dim_payroll[:2]

,EmployeeID,ReviewPeriod,Rating,Comments,Prediction
0,102,Q2 2023,2.6,Good performance,SAMPLE
1,103,Q3 2023,1.5,Good performance,SAMPLE


In [ ]:
dim_payroll_query = read_sql_query('dim_payroll_query.sql')
print('[INFO] service etl dim_payroll is running...')

# Data fact_employee_query ETL
print('[INFO] Fetching data for dim_payroll...')
df_dim_payroll = pd.read_sql(dim_payroll_query, engine_dwh)
print(df_dim_payroll.head(5))
# Insert data
error_status = insert_into_table(conn_dwh, df_dim_payroll, 'dim_payroll')

# Close connection
conn_dwh.close()

In [ ]:
etl_process_postgresql_dim(query_file, table):
    dim_query = read_sql_query(query_file)
    print(f"[INFO] service etl {query_file} is running...")
    
    # Data fact_employee_query ETL
    print(f"[INFO] Fetching data for {query_file}...")
    df = pd.read_sql(dim_query, engine_dwh)
    print(df.head(5))
    # Insert data
    error_status = insert_into_table(conn_dwh, df, table)
    
    # Close connection
    conn_dwh.close()

In [74]:
pip install pyspark


Note: you may need to restart the kernel to use updated packages.


In [75]:
pip install notebook


  Using cached fqdn-1.5.1-py3-none-any.whl.metadata (1.4 kB)
  Using cached isoduration-20.11.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached uri_template-1.3.0-py3-none-any.whl.metadata (8.8 kB)
  Using cached webcolors-24.11.1-py3-none-any.whl.metadata (2.2 kB)
Using cached webcolors-24.11.1-py3-none-any.whl (14 kB)
Using cached fqdn-1.5.1-py3-none-any.whl (9.1 kB)
Using cached isoduration-20.11.0-py3-none-any.whl (11 kB)
Using cached uri_template-1.3.0-py3-none-any.whl (11 kB)
Note: you may need to restart the kernel to use updated packages.
